In [124]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [125]:
test_private=pd.read_csv('../../data/private_x_alert_date.csv')

In [126]:
private_list=test_private.alert_key.tolist()

In [127]:
df_=pd.read_csv('../output/final_output.csv')

In [128]:
df_

,csv,seed,cols,lgb,xgb,xgb2,blender,blender2,stacker,stacker_n,stacker2
0,df_train_290.csv,42,43,0,0,0,0,0,0,0,0.030488
1,df_train_280.csv,42,43,0,0,0,0,0,0,0,0.010917
2,df_train_290.csv,142,43,0,0,0,0,0,0,0,0.025316
3,df_train_280.csv,142,43,0,0,0,0,0,0,0,0.012626
4,df_train_290.csv,242,43,0,0,0,0,0,0,0,0.013774
...,...,...,...,...,...,...,...,...,...,...,...
95,df_train_280.csv,2721,43,0,0,0,0,0,0,0,0.023256
96,df_train_290.csv,3721,43,0,0,0,0,0,0,0,0.013055
97,df_train_280.csv,3721,43,0,0,0,0,0,0,0,0.013986
98,df_train_290.csv,4721,43,0,0,0,0,0,0,0,0.008482


In [129]:
file_list=[]
for i, row in df_.iterrows():
    if row['stacker2']>=0.015:
        file_list.append( f"{row['seed']}_stacker2_{row['csv']}")
    if row['blender']>=0.015:
        file_list.append(f"{row['seed']}_blender_{row['csv']}")    

In [130]:
folder='../output/'

In [131]:
file_cnt=len(file_list)
file_cnt

57

In [132]:
col_list=['prob','rank']
res_list=['alert_key']
for i in range(1):
    for col in col_list:
        res_list.append(f'{col}_{i}')

In [133]:
df_ = pd.DataFrame(columns=res_list)

In [134]:
for i,file in enumerate(file_list):
    if i==0:
        print(file)
        tmp_df = pd.read_csv(folder+file)
        tmp_df.columns=['alert_key','prob_0']
        tmp_df=tmp_df.sort_values('prob_0',ascending = False).reset_index(drop=True)
        tmp_df['rank_0']=tmp_df.index+1
        df_=df_.append(tmp_df)
    else:
        tmp_df = pd.read_csv(folder+file)
        tmp_df.columns=['alert_key','prob_'+str(i)]
        tmp_rk_df=tmp_df.sort_values(f'prob_{i}',ascending = False).reset_index(drop=True)
        tmp_rk_df['rank_'+str(i)]=tmp_rk_df.index+1
        df_ = pd.merge(df_,tmp_rk_df,on=['alert_key'],how='left')

42_stacker2_df_train_290.csv


In [135]:
rank_list=df_['rank_0'].values.tolist()
for i in range(1,file_cnt):
    rank_list = (np.array(rank_list) + np.array(df_['rank_'+str(i)].values.tolist())).tolist()

In [136]:
df_['rank_sum']=rank_list

### Rank sum

In [137]:
df_=df_.sort_values('rank_sum',ascending = True).reset_index(drop=True)

In [138]:
df_r=df_[~df_.alert_key.isin(private_list)].sort_values('rank_sum',ascending = True).reset_index(drop=True)
df_r2=df_.sort_values('rank_sum',ascending = True).reset_index(drop=True)

In [139]:
df_['rank_idx']=df_.index

In [140]:
df_['rank_score']= df_['rank_idx'].apply(lambda x: (3850-x)/3851)

In [141]:
df_r2['rank_idx'] = df_r2.index

In [142]:
df_r2['rank_score'] = df_r2['rank_idx'].apply(lambda x: (3850-x)/3851)

In [143]:
submit=df_r2[['alert_key','rank_score']]
submit.columns=['alert_key','probability']

In [144]:
submit.to_csv('../output/final_rank_sum.csv', index=False)

In [90]:
y=[
354939,
355091,
355152,
355724,
359668,
356602,
363320,
358453,
363896,
361617,
363033]

In [146]:
df_[df_.alert_key.isin(y)]

,alert_key,prob_0,rank_0,prob_1,rank_1,prob_2,rank_2,prob_3,rank_3,prob_4,...,rank_53,prob_54,rank_54,prob_55,rank_55,prob_56,rank_56,rank_sum,rank_idx,rank_score
14,358453,0.5127,20,0.2739,72,0.5032,18,0.4040,28,0.4035,...,51,0.2128,100,0.4002,27,0.3939,35,2377,14,0.996105
22,361617,0.2651,70,0.2226,91,0.2006,80,0.5408,16,0.3292,...,62,0.5282,18,0.5029,14,0.4966,22,2709,22,0.994028
44,363033,0.5298,18,0.1807,114,0.1378,104,0.4032,29,0.1515,...,107,0.6956,6,0.3212,57,0.8614,2,4138,44,0.988315
99,356602,0.2081,85,0.3599,50,0.1623,88,0.0576,219,0.2046,...,147,0.4632,27,0.1183,150,0.2957,60,6951,99,0.974033
102,355724,0.0910,162,0.0605,198,0.3258,40,0.2966,47,0.1677,...,80,0.1331,137,0.2185,100,0.1308,142,7035,102,0.973254
112,363896,0.0413,230,0.0563,206,0.0411,222,0.2236,71,0.2176,...,163,0.0953,164,0.0723,191,0.0679,200,7620,112,0.970657
189,359668,0.0732,177,0.0375,251,0.0256,270,0.0651,203,0.0305,...,169,0.0667,201,0.0462,229,0.0649,207,13201,189,0.950662
223,354939,0.0435,225,0.3606,49,0.0074,499,0.0279,298,0.2591,...,95,0.0270,299,0.2156,103,0.0126,424,15699,223,0.941833
232,355091,0.0687,182,0.0146,395,0.0146,351,0.0359,271,0.1125,...,270,0.0120,428,0.1024,162,0.0316,288,16308,232,0.939496
338,355152,0.0176,327,0.0614,194,0.0071,510,0.0464,244,0.0056,...,402,0.0089,493,0.0049,660,0.0123,430,23830,338,0.911971


In [102]:
df_r2_=df_r2[~df_r2.alert_key.isin(private_list)].sort_values('rank_sum',ascending = True).reset_index(drop=True)

In [107]:
df__=df_r2_[df_r2_.alert_key.isin(y)]

In [148]:
10/120, 10/218, 20/338

(0.08333333333333333, 0.045871559633027525, 0.05917159763313609)

In [108]:
print(df__.index)

Int64Index([4, 6, 13, 31, 32, 35, 60, 74, 78, 120, 810], dtype='int64')
